# Projet : Detection d’Anomalies dans le Trafic Reseau avec LightGBM

In [2]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('data-20221207.csv')

In [7]:
df.shape

(35250, 84)

In [ ]:
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, train_test_split
import lightgbm as lgb
import optuna

In [ ]:
# Séparation des données en entraînement et test
X_entrainement, X_test, y_entrainement, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Pipeline avec SMOTE et LightGBM
pipeline = ImbPipeline(steps=[
    ('equilibrage_classes', SMOTE(sampling_strategy='auto', random_state=42)),  # Gestion du déséquilibre des classes
    ('modele', lgb.LGBMClassifier())  # Modèle LightGBM
])


In [ ]:
# Optimisation des hyperparamètres avec GridSearchCV
grille_parametres = {
    'modele__num_leaves': [31, 50, 70],
    'modele__learning_rate': [0.01, 0.1, 0.2],
    'modele__n_estimators': [100, 200, 300]
}

recherche_grille = GridSearchCV(pipeline, grille_parametres, cv=5, scoring='f1', verbose=2, n_jobs=-1)
recherche_grille.fit(X_entrainement, y_entrainement)

print("Meilleurs paramètres GridSearchCV :", recherche_grille.best_params_)


In [ ]:
# Optimisation des hyperparamètres avec Optuna
def fonction_objectif(essai):
    parametres = {
        'num_leaves': essai.suggest_int('num_leaves', 20, 100),
        'learning_rate': essai.suggest_loguniform('learning_rate', 0.001, 0.1),
        'n_estimators': essai.suggest_int('n_estimators', 50, 300)
    }
    modele_lightgbm = lgb.LGBMClassifier(**parametres)
    
    pipeline_optuna = ImbPipeline(steps=[
        ('equilibrage_classes', SMOTE(sampling_strategy='auto', random_state=42)),  # SMOTE
        ('modele', modele_lightgbm)  # Modèle optimisé
    ])
    
    pipeline_optuna.fit(X_entrainement, y_entrainement)
    return pipeline_optuna.score(X_test, y_test)

etude = optuna.create_study(direction='maximize')
etude.optimize(fonction_objectif, n_trials=20)

print("Meilleurs paramètres Optuna :", etude.best_params)